***1. Задание:***

login — логин асессора;
tid — id оцениваемого задания (task id);
Microtasks – количество микрозаданий в одном задании;
assigned_ts — время резервирования системой задания для асессора;
closed_ts — точное время завершения работы над заданием;
разделитель — табуляция \t.

Задание может состоять из одного или несколько микрозаданий.

Время резервирования задания (assigned_ts) указывает на тот момент, когда система назначила определенного асессора исполнителем этого задания.

Этот момент может совпадать с временем начала работы асессора над заданием, а может и не совпадать

(асессор может отойти выпить чаю, а потом приступить к заданию, асессор может выполнять предыдущее задание, в то время как за ним зарезервированы новые).

Предположим, что асессор за 30 секунд своего рабочего времени получает N рублей.

Какую оплату вы считаете справедливой для выполнения асессором одного микрозадания из этого файла?

In [1]:
import pandas as pd
import numpy as np

%pylab inline
%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pd.read_csv('/Users/kateosti/Downloads/data_task4_old.txt', sep='\t', index_col=False)

### 1. Обработка данных

In [3]:
data.head(8) # Посмотрим данные

,login,tid,Microtasks,assigned_ts,closed_ts
0,login0,190563850.0,4.0,2017-04-20 12:09:39,2017-04-20 13:13:01
1,login0,190561754.0,1.0,2017-04-20 12:10:30,2017-04-20 12:28:29
2,login0,190565906.0,4.0,2017-04-20 12:21:31,2017-04-20 13:30:10
3,login0,190560246.0,1.0,2017-04-20 12:28:30,2017-04-20 13:13:56
4,login0,190562168.0,2.0,2017-04-20 12:28:42,2017-04-20 13:14:50
5,login0,190560934.0,1.0,2017-04-20 12:28:53,2017-04-20 13:15:23
6,login0,190563590.0,1.0,2017-04-20 12:29:06,2017-04-20 13:15:36
7,login0,190565094.0,1.0,2017-04-20 13:47:43,2017-04-20 14:19:46


In [4]:
data.info() # Общая информация и увидим есть ли пропуски, их нету

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701827 entries, 0 to 701826
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   login        701827 non-null  object 
 1   tid          701827 non-null  float64
 2   Microtasks   701827 non-null  float64
 3   assigned_ts  701827 non-null  object 
 4   closed_ts    701827 non-null  object 
dtypes: float64(2), object(3)
memory usage: 26.8+ MB


In [5]:
data.describe() # Посмотрим кратко на основные статические данные

,tid,Microtasks
count,7.018270e+05,701827.00000
mean,1.884911e+08,3.25439
std,1.391113e+07,3.65397
min,1.139586e+08,1.00000
25%,1.906846e+08,1.00000
50%,1.935373e+08,2.00000
75%,1.963667e+08,4.00000
max,1.985758e+08,50.00000


In [6]:
# Приведем данные к нужным типам данных: из обжект в даты, из флоат к целым значениям

data['assigned_ts'] = pd.to_datetime(data['assigned_ts'], errors='coerce')
data['closed_ts'] = pd.to_datetime(data['closed_ts'], errors='coerce')
data['tid'] = data['tid'].astype('int64')
data['Microtasks'] = data['Microtasks'].astype('int64')

In [7]:
data.dtypes # Проверим типы данных

login                  object
tid                     int64
Microtasks              int64
assigned_ts    datetime64[ns]
closed_ts      datetime64[ns]
dtype: object

In [8]:
# так как логин у нас один и тот же, роли не играет, можно его вырезать из датасета
data.pop('login')
data

,tid,Microtasks,assigned_ts,closed_ts
0,190563850,4,2017-04-20 12:09:39,2017-04-20 13:13:01
1,190561754,1,2017-04-20 12:10:30,2017-04-20 12:28:29
2,190565906,4,2017-04-20 12:21:31,2017-04-20 13:30:10
3,190560246,1,2017-04-20 12:28:30,2017-04-20 13:13:56
4,190562168,2,2017-04-20 12:28:42,2017-04-20 13:14:50
...,...,...,...,...
701822,195656026,1,2017-05-15 15:53:27,2017-05-15 16:16:56
701823,195656174,1,2017-05-15 15:53:33,2017-05-15 16:21:53
701824,195656466,1,2017-05-15 15:53:38,2017-05-15 16:23:41
701825,195656336,3,2017-05-15 15:54:18,2017-05-15 16:32:11


### 2. Выполнение задания.

2.1 Посчитаем время выполнения каждого задания

In [9]:
from datetime import *
# отнимем от времения окончания работы -  время начала работы
data['time_tsk'] = data['closed_ts'] -  data['assigned_ts']

In [21]:
print("Среднее время выполнения задания", data['time_tsk'].mean())
print("Медианное время выполнения задания", data['time_tsk'].median())

Среднее время выполнения задания 0 days 00:32:09.659941
Медианное время выполнения задания 0 days 00:02:37


In [19]:
data['time_tsk'].sort_values(ascending = False) # Посмотрим примерно сколько по времени выполняют ассесоры задания

603896    23 days 15:32:58
603555    23 days 15:32:58
291798    23 days 09:48:32
292123    23 days 09:48:32
292126    22 days 09:51:02
                ...       
640216     0 days 00:00:03
640206     0 days 00:00:03
640204     0 days 00:00:03
640201     0 days 00:00:03
534453   -1 days +23:47:51
Name: time_tsk, Length: 701827, dtype: timedelta64[ns]

In [20]:
data['sec_ts'] = data['time_tsk'].astype('timedelta64[s]') # переведем данные в секунды

In [15]:
# Посчитаем деньги, каждые 30 секунд стоят N. За N возьмем 1.
N = 1
t = 30
data['money'] = round(data['sec_ts'] / 30)
data

,tid,Microtasks,assigned_ts,closed_ts,time_tsk,sec_ts,money
0,190563850,4,2017-04-20 12:09:39,2017-04-20 13:13:01,01:03:22,3802.0,127.0
1,190561754,1,2017-04-20 12:10:30,2017-04-20 12:28:29,00:17:59,1079.0,36.0
2,190565906,4,2017-04-20 12:21:31,2017-04-20 13:30:10,01:08:39,4119.0,137.0
3,190560246,1,2017-04-20 12:28:30,2017-04-20 13:13:56,00:45:26,2726.0,91.0
4,190562168,2,2017-04-20 12:28:42,2017-04-20 13:14:50,00:46:08,2768.0,92.0
...,...,...,...,...,...,...,...
701822,195656026,1,2017-05-15 15:53:27,2017-05-15 16:16:56,00:23:29,1409.0,47.0
701823,195656174,1,2017-05-15 15:53:33,2017-05-15 16:21:53,00:28:20,1700.0,57.0
701824,195656466,1,2017-05-15 15:53:38,2017-05-15 16:23:41,00:30:03,1803.0,60.0
701825,195656336,3,2017-05-15 15:54:18,2017-05-15 16:32:11,00:37:53,2273.0,76.0


In [16]:
data['price_Mikr'] = data['money'] / data['Microtasks']

In [17]:
data

,tid,Microtasks,assigned_ts,closed_ts,time_tsk,sec_ts,money,price_Mikr
0,190563850,4,2017-04-20 12:09:39,2017-04-20 13:13:01,01:03:22,3802.0,127.0,31.750000
1,190561754,1,2017-04-20 12:10:30,2017-04-20 12:28:29,00:17:59,1079.0,36.0,36.000000
2,190565906,4,2017-04-20 12:21:31,2017-04-20 13:30:10,01:08:39,4119.0,137.0,34.250000
3,190560246,1,2017-04-20 12:28:30,2017-04-20 13:13:56,00:45:26,2726.0,91.0,91.000000
4,190562168,2,2017-04-20 12:28:42,2017-04-20 13:14:50,00:46:08,2768.0,92.0,46.000000
...,...,...,...,...,...,...,...,...
701822,195656026,1,2017-05-15 15:53:27,2017-05-15 16:16:56,00:23:29,1409.0,47.0,47.000000
701823,195656174,1,2017-05-15 15:53:33,2017-05-15 16:21:53,00:28:20,1700.0,57.0,57.000000
701824,195656466,1,2017-05-15 15:53:38,2017-05-15 16:23:41,00:30:03,1803.0,60.0,60.000000
701825,195656336,3,2017-05-15 15:54:18,2017-05-15 16:32:11,00:37:53,2273.0,76.0,25.333333


In [18]:
print("Средняя цена за Микрозадание", data['price_Mikr'].mean())
print("Медианная цена за Микрозадание", data['price_Mikr'].median())

Средняя цена за Микрозадание 18.941449606544936
Медианная цена за Микрозадание 2.5


### Вывод: 
#### если ассесор делает одно Микрозадание в среднем 32:09 минуты, тогда он получит 19N,
#### А в общем ассесор делает задание почти 3 минуты, тогда он получит 2,5N